In [3]:
! pip install datasets transformers nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [4]:
from transformers import (AutoModelForSeq2SeqLM,
                            AutoTokenizer,
                            T5Tokenizer)
import torch
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
from tqdm import tqdm

#import subprocess
import sys
import os
import argparse
from IPython.core import error
import random
import numpy as np
import nltk
import json
import csv

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
output_dir = '/content/drive/MyDrive/'
# Here you need to link this path in your Google drive to the place preseving your model weights, e.g., checkpoint-62500
# You can download it on the github page

model_checkpoint = "t5-large"
prefix = "Transform the following sentence into Signal Temporal logic: "

max_input_length = 1024
max_target_length = 128
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=max_input_length)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tl_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir+"t5-large-navi-epoch20-trainpoint3").to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
import time
time_start = time.time()
inputs = [prefix + 'At some point (prop_1), and at some point (prop_2), and always do not (prop_4).']
inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(decoded_output)
time_end = time.time()
print('Translation time: ', time_end-time_start)

( ( prop_1 and prop_2 ) and globally ( negation prop_4 ) )
Translation time:  1.7932915687561035


In [ ]:
# Here are the example test sentences
test_sentence = ['Stay at (prop_1) for 5 units in the future and stay at (prop_2) for 5 units in the future, and ensure that never (prop_3).',
                 'First (prop_1), and then (prop_2), and ensure that never (prop_3).',
                 'Start by (prop_1). Then, (prop_2). Lastly, (prop_3).',
                 'Guarantee that you (prop_1) and (prop_2)', # Input the natural sentence
                 '( prop_1 ) and whenever ( prop_2 )',
                 'Sooner or later (prop_1)',
                 'Repeatedly (prop_1)',
                 'At some point, (prop_1).',
                 'Do prop_1 but not do prop_2',
                 'Do prop_1, do prop_2, do prop_3'] # Input the natural sentence

for sentence in test_sentence:
  inputs = [prefix + sentence]
  inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
  output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  print('Input sentence: ', sentence)
  print('Translated STL: ', decoded_output)
  print('\n')

Input sentence:  Stay at (prop_1) for 5 units in the future and stay at (prop_2) for 5 units in the future, and ensure that never (prop_3).
Translated STL:  ( ( globally [0,5] prop_1 and globally [0,5] prop_2 ) and globally [0,1] negation prop_3 )


Input sentence:  First (prop_1), and then (prop_2), and ensure that never (prop_3).
Translated STL:  ( ( prop_1 until prop_2 ) and globally ( negation prop_3 ) )


Input sentence:  Start by (prop_1). Then, (prop_2). Lastly, (prop_3).
Translated STL:  ( ( prop_1 until prop_2 ) until prop_3 )


Input sentence:  Guarantee that you (prop_1) and (prop_2)
Translated STL:  finally ( prop_1 and prop_2 )


Input sentence:  ( prop_1 ) and whenever ( prop_2 )
Translated STL:  ( prop_1 and finally prop_2 )


Input sentence:  Sooner or later (prop_1)
Translated STL:  finally prop_1


Input sentence:  Repeatedly (prop_1)
Translated STL:  globally ( finally prop_1 )


Input sentence:  At some point, (prop_1).
Translated STL:  finally prop_1


Input senten

In [9]:
# Here are the example test sentences
test_sentence = ['At some time travel house or at some time go to apple and pick it'] # Input the natural sentence
#test_sentence = ['whenever stop by house or at some time make to flag.','At some time approach trash can or whenever got to house.','Whenever travel house or at any time go to flag.']
for sentence in test_sentence:
  inputs = [prefix + sentence]
  inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
  output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  print('Input sentence: ', sentence)
  print('Translated STL: ', decoded_output)
  print('\n')

Input sentence:  At some time travel house or at some time go to apple and pick it
Translated STL:  finally ( travel_v house_n ) or finally ( go_to_v apple_n ) and pick_it_n




In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 859.2 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2


In [8]:
import openai
from pyparsing import (
    infixNotation, opAssoc, Keyword, ParserElement, ParseException,
    Regex, Suppress, Optional, Group, Word, nums, OneOrMore, Combine
)
OPENAI_API_KEY = "Your-API-KEY"
openai.api_key = OPENAI_API_KEY
# 启用 packrat 加速解析
ParserElement.enablePackrat()

# 定义区间表达式，例如 [0,5]
interval = Group(
    Suppress("[") + Word(nums)("start") + Suppress(",") + Word(nums)("end") + Suppress("]")
)

# 定义保留词（操作符）
imply_kw    = Keyword("imply")
equal_kw    = Keyword("equal")
and_kw      = Keyword("and")
or_kw       = Keyword("or")
finally_kw  = Keyword("finally")
globally_kw = Keyword("globally")
until_kw    = Keyword("until")
negation_kw = Keyword("negation")

# 辅助函数：整合带可选区间的操作符
def flatten_operator(tokens):
    op = tokens[0]
    if len(tokens) > 1 and tokens[1]:
        intv = tokens[1]
        return f"{op}[{intv['start']},{intv['end']}]"
    else:
        return op

# 定义 temporal 操作符支持可选区间
globally_temporal = (globally_kw + Optional(interval)).setParseAction(flatten_operator)
finally_temporal  = (finally_kw + Optional(interval)).setParseAction(flatten_operator)
until_temporal    = (until_kw + Optional(interval)).setParseAction(flatten_operator)

# 其他操作符直接用
imply_op = imply_kw
equal_op = equal_kw
and_op   = and_kw
or_op    = or_kw
negation_op = negation_kw

# 定义 AP (原子命题)
AP = Regex(
    r"(?!^(?:imply|equal|and|or|finally|globally|until|negation)$)[A-Za-z0-9_ ]+"
).setName("AP").addParseAction(lambda t: t[0].strip())

# 构造表达式：采用 infixNotation 定义一元和二元运算符
expr = infixNotation(
    AP,
    [
        (negation_op, 1, opAssoc.RIGHT),
        ((globally_temporal | finally_temporal), 1, opAssoc.RIGHT),
        (until_temporal, 2, opAssoc.RIGHT),
        ((and_op | or_op | imply_op | equal_op), 2, opAssoc.RIGHT),
    ]
)

def is_unary_op(token):
    for op in ["negation", "globally", "finally"]:
        if token.startswith(op):
            return True
    return False

def is_binary_op(token):
    for op in ["imply", "equal", "and", "or", "until"]:
        if token.startswith(op):
            return True
    return False

def check_tree(tree):
    if isinstance(tree, str):
        return bool(tree.strip())
    if isinstance(tree, list):
        if len(tree) == 1:
            return check_tree(tree[0])
        if len(tree) == 2:
            op, operand = tree
            if isinstance(op, str) and is_unary_op(op):
                return check_tree(operand)
            return False
        if len(tree) == 3:
            left, op, right = tree
            if isinstance(op, str) and is_binary_op(op):
                return check_tree(left) and check_tree(right)
            return False
        return False
    return False

# 验证 STL 表达式函数：返回 (True, "") 或 (False, error_message)
def validate_stl(stl_expression):
    try:
        result = expr.parseString(stl_expression, parseAll=True)
        tree = result.asList()
        if check_tree(tree):
            return (True, "")
        else:
            return (False, "Structure check failed")
    except ParseException as pe:
        return (False, str(pe))

# 自动修正 STL 的函数，包含正式规则说明（在 prompt 中传递）
def auto_correct_stl(nl, stl, error_message):
    # 这里你可以在 prompt 中附加详细的正式规则说明
    formal_rules = (
        "正式规则：\n"
        "1. 每个二元操作符必须有两个操作数，并且左右操作数必须用括号包围。\n"
        "2. 时间区间必须紧跟在 temporal 操作符后面，格式为 [start,end]，后面必须跟括号包围的子公式。\n"
        "3. 括号必须完全匹配。\n"
        "请将错误的 STL 修正为符合上述规则的正确 STL 表达式，且保持所有操作符原样。"
    )
    prompt = (
        f"原自然语言描述: {nl}\n"
        f"错误的 STL: {stl}\n"
        f"解析错误信息: {error_message}\n"
        f"{formal_rules}\n"
        "请仅输出修正后的 STL 表达式，不附带其他解释。"
    )
    try:
        response = openai.ChatCompletion.create(
            model='o3-mini',
            messages=[
                {"role": "system",
                 "content": "你是一个时序逻辑专家，擅长根据正式规则修正 STL 表达式。请将错误 STL 修正为既符合语法又符合语义的正确 STL 表达式，且保持所有操作符的原始英文单词。"},
                {"role": "user", "content": prompt}
            ],
            reasoning_effort='high'
        )
        corrected = response.choices[0].message['content'].strip()
        return corrected
    except Exception as e:
        print("OpenAI API error:", e)
        return stl

# ===============================
# 流程集成：在模型翻译前先进行共指消解，然后翻译 STL，再检测 STL 是否符合正式规则，
# 如有错误则调用 LLM 自动修正
# ===============================

def contains_pronoun(sentence):
    tokens = sentence.lower().split()
    return any(p in tokens for p in ("it", "them"))

def resolve_coreference(nl_sentence):
    prompt = (
        f"请将下面句子中的代词（例如 'it', 'them' 等）替换为它们所指代的具体名词短语，消除共指：\n"
        f"{nl_sentence}\n"
        "请仅输出处理后的句子，不附带其他说明。"
    )
    try:
        response = openai.ChatCompletion.create(
            model='o3-mini',
            messages=[
                {"role": "system", "content": "你是一个处理共指问题的语言学专家。"},
                {"role": "user", "content": prompt}
            ],
            reasoning_effort='high'
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        print("OpenAI API error during coreference resolution:", e)
        return nl_sentence

# API_MODE 控制是否总调用自动修正模块（"always" 或 "only_if_error"）
API_MODE = "only_if_error"

# 假定以下变量已定义：prefix, tokenizer, tl_model, max_input_length, max_target_length, device
# 下面是集成调用示例：
#test_sentence = ['whenever stop by house or at some time make to flag.','At some time approach trash can or whenever got to house.','Whenever travel house or at any time go to flag.']
#test_sentence = ['whenever stop by house, make to flag.']
test_sentence = ['At some time travel house or at some time go to apple and pick it']
for sentence in test_sentence:
    # 共指检测与消解
    if contains_pronoun(sentence):
        resolved_sentence = resolve_coreference(sentence)
        sentence = resolved_sentence  # 用消解后的句子
        print("Coreference resolved sentence:", sentence)

    # 翻译部分（注意：确保 prefix、tokenizer、tl_model、device 等变量已定义）
    inputs = [prefix + sentence]
    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
    output = tl_model.generate(**inputs, num_beams=8, do_sample=True, max_length=max_target_length)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

    # STL 验证
    valid, error_message = validate_stl(decoded_output)
    if API_MODE == "always" or (API_MODE == "only_if_error" and not valid):
        final_stl = auto_correct_stl(sentence, decoded_output, error_message)
    else:
        final_stl = decoded_output

    # 简洁输出
    print("Input sentence: ", sentence)
    print("Translated STL: ", decoded_output)
    print("Final STL: ", final_stl)
    print('\n')

Coreference resolved sentence: At some time travel house or at some time go to apple and pick apple.
Input sentence:  At some time travel house or at some time go to apple and pick apple.
Translated STL:  finally ( travel_v house_n ) or finally ( go_to_v apple_n ) and pick_v apple_n
Final STL:  finally ( travel_v house_n ) or finally ( go_to_v apple_n ) and pick_v apple_n


